# データの準備

In [8]:
import sys
print(sys.version)

3.8.10 (default, Mar 15 2022, 12:22:08) 
[GCC 9.4.0]


In [9]:
import os
import urllib.request
import tarfile
import zipfile
from tqdm import tqdm

In [10]:
# フォルダ「data」が存在しない場合は作成する
data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

# tar.gzの解凍
# tarファイルを読み込み

tar = tarfile.open('./data/stair_captions_v1.2.tar.gz')
tar.extractall('./data/')  # 解凍
tar.close()  # ファイルをクローズ



In [11]:
# MSCOCOの2014 Val images [41K/6GB]をダウンロード
# 6GBのダウンロードと解凍なので時間がかかります（10分弱）

url =  "http://images.cocodataset.org/zips/val2014.zip"
target_path = os.path.join(data_dir, "val2014.zip") 

if not os.path.exists(target_path):
    urllib.request.urlretrieve(url, target_path)

zip = zipfile.ZipFile(target_path)
zip.extractall(data_dir)  # ZIPを解凍
zip.close()  # ZIPファイルをクローズ

In [12]:
import json
import pandas as pd
import numpy as np
 
f = open("./data/stair_captions_v1.2_val_tokenized.json", 'r', encoding='utf-8')
json_data = json.load(f)

In [13]:
id_file = {}
for i in range(len(json_data["images"])):
    file_name = json_data["images"][i]["file_name"]
    id = json_data["images"][i]["id"]
    id_file[id] = file_name

In [14]:
annotations_size = len(json_data["annotations"])
df = pd.DataFrame(data = np.zeros((annotations_size,2)))

for j in tqdm(range(annotations_size)):
    image_id = json_data["annotations"][j]['image_id']
    file_name = id_file[image_id]
    caption = json_data["annotations"][j]["tokenized_caption"]
    df.loc[j, 0] = file_name
    df.loc[j, 1] = caption

100%|███████████████████████████████████| 202520/202520 [34:18<00:00, 98.37it/s]


In [15]:
data_size = 202520
df.sort_values(0, inplace=True)
df= df.reset_index(drop=True)
df= df.loc[:data_size-1]

#txtファイルに保存
df.to_csv("./data/caption_val.txt", sep=" ", header=False, index=False, encoding='utf-8')

In [17]:
print()

画像のファイル名とキャプションがセットになったtxtファイルの作成が完了